# Pyspark for RandomForest Regression

### 1. Set up spark context and SparkSession

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark RandomForest Regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### 2. Load dataset

In [2]:
df = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').load("../data/Advertising.csv",header=True);

In [3]:
df.show(5,True)
df.printSchema()

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



In [4]:
df.count()

200

In [5]:
# # Convert to float format
# def string_to_float(x):
#     return float(x)


# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType, DoubleType
# string_to_float_udf = udf(string_to_float, DoubleType())

# df = df.withColumn("horsepower", string_to_float_udf("horsepower"))

In [6]:
df.show(5,True)
df.printSchema()

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



In [7]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|               0.0|               0.3|               1.6|
|    max|            296.4|              49.6|             114.0|              27.0|
+-------+-----------------+------------------+------------------+------------------+



### 3. Convert the data to dense vector

- check the dataset and the Schema

In [8]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [9]:
# convert the data to dense vector
#def transData(row):
#    return Row(label=row["Sales"],
#               features=Vectors.dense([row["TV"],
#                                       row["Radio"],
#                                       row["Newspaper"]]))
def transData(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])

### 4. Transform the dataset to DataFrame

In [10]:
#transformed = df.rdd.map(transData).toDF() 
transformed= transData(df)

In [11]:
transformed.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[230.1,37.8,69.2]| 22.1|
| [44.5,39.3,45.1]| 10.4|
| [17.2,45.9,69.3]|  9.3|
|[151.5,41.3,58.5]| 18.5|
|[180.8,10.8,58.4]| 12.9|
+-----------------+-----+
only showing top 5 rows



### 5. Deal With Categorical Variables 

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [13]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures",\
                               maxCategories=4).fit(transformed)

In [14]:
data = featureIndexer.transform(transformed)

In [15]:
data.show(5,True)

+-----------------+-----+-----------------+
|         features|label|  indexedFeatures|
+-----------------+-----+-----------------+
|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----------------+-----+-----------------+
only showing top 5 rows



### 6. Split the data to training and test data sets

In [16]:
# Split the data into training and test sets (40% held out for testing)
(trainingData, testData) = data.randomSplit([0.6, 0.4])

In [17]:
trainingData.show(5)
testData.show(5)

+----------------+-----+----------------+
|        features|label| indexedFeatures|
+----------------+-----+----------------+
|  [4.1,11.6,5.7]|  3.2|  [4.1,11.6,5.7]|
|  [5.4,29.9,9.4]|  5.3|  [5.4,29.9,9.4]|
|   [8.6,2.1,1.0]|  4.8|   [8.6,2.1,1.0]|
| [13.1,0.4,25.6]|  5.3| [13.1,0.4,25.6]|
|[17.9,37.6,21.6]|  8.0|[17.9,37.6,21.6]|
+----------------+-----+----------------+
only showing top 5 rows

+---------------+-----+---------------+
|       features|label|indexedFeatures|
+---------------+-----+---------------+
| [0.7,39.6,8.7]|  1.6| [0.7,39.6,8.7]|
|[7.3,28.1,41.4]|  5.5|[7.3,28.1,41.4]|
|[7.8,38.9,50.6]|  6.6|[7.8,38.9,50.6]|
| [8.4,27.2,2.1]|  5.7| [8.4,27.2,2.1]|
|[8.7,48.9,75.0]|  7.2|[8.7,48.9,75.0]|
+---------------+-----+---------------+
only showing top 5 rows



### 7. Fit RandomForest Regression Model

In [18]:
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol="indexedFeatures", maxIter=100)

### 8. Pipeline Architecture

In [19]:
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

In [20]:
model = pipeline.fit(trainingData)

### 9. Summary of the Model

In [21]:
# def modelsummary(model):
#     import numpy as np
#     print ("Note: the last rows are the information for Intercept")
#     print ("##","-------------------------------------------------")
#     print ("##","  Estimate   |   Std.Error | t Values  |  P-value")
#     coef = np.append(list(model.coefficients),model.intercept)
#     Summary=model.summary
    
#     for i in range(len(Summary.pValues)):
#         print ("##",'{:10.6f}'.format(coef[i]),\
#         '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
#         '{:8.3f}'.format(Summary.tValues[i]),\
#         '{:10.6f}'.format(Summary.pValues[i]))
        
#     print ("##",'---')
#     print ("##","Mean squared error: % .6f" \
#            % Summary.meanSquaredError, ", RMSE: % .6f" \
#            % Summary.rootMeanSquaredError )
#     print ("##","Multiple R-squared: %f" % Summary.r2, ", \
#             Total iterations: %i"% Summary.totalIterations)    

### 10. Make predictions

In [22]:
predictions = model.transform(testData)

In [23]:
# Select example rows to display.
predictions.select("features","label", "prediction").show(5)

+---------------+-----+-----------------+
|       features|label|       prediction|
+---------------+-----+-----------------+
| [0.7,39.6,8.7]|  1.6|5.322928182336102|
|[7.3,28.1,41.4]|  5.5|5.182267596482987|
|[7.8,38.9,50.6]|  6.6|5.019960839826547|
| [8.4,27.2,2.1]|  5.7|3.134398824899578|
|[8.7,48.9,75.0]|  7.2|6.081063813160135|
+---------------+-----+-----------------+
only showing top 5 rows



### 11. Evaluation

In [24]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 1.33595


In [25]:
y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

In [26]:
import sklearn.metrics 
r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {:4.3f}'.format(r2_score)) 

r2_score: 0.937


In [27]:
model.stages[-1].featureImportances

SparseVector(3, {0: 0.3162, 1: 0.3905, 2: 0.2933})

In [28]:
model.stages[-1].trees

[DecisionTreeRegressionModel (uid=dtr_6ecfcef81b52) of depth 5 with 61 nodes,
 DecisionTreeRegressionModel (uid=dtr_7b7490a8babc) of depth 5 with 37 nodes,
 DecisionTreeRegressionModel (uid=dtr_a1ef21bc4763) of depth 5 with 39 nodes,
 DecisionTreeRegressionModel (uid=dtr_f2a5665d36e6) of depth 5 with 33 nodes,
 DecisionTreeRegressionModel (uid=dtr_0d78f83b525e) of depth 5 with 39 nodes,
 DecisionTreeRegressionModel (uid=dtr_d8249c9f6871) of depth 5 with 43 nodes,
 DecisionTreeRegressionModel (uid=dtr_b5f7f6e44748) of depth 5 with 49 nodes,
 DecisionTreeRegressionModel (uid=dtr_75b36bc4b0f7) of depth 5 with 51 nodes,
 DecisionTreeRegressionModel (uid=dtr_9e32b04d48a7) of depth 5 with 57 nodes,
 DecisionTreeRegressionModel (uid=dtr_ee1b0eb4a5f9) of depth 5 with 45 nodes,
 DecisionTreeRegressionModel (uid=dtr_9f025eee9842) of depth 5 with 51 nodes,
 DecisionTreeRegressionModel (uid=dtr_9d1c2861bb1c) of depth 5 with 55 nodes,
 DecisionTreeRegressionModel (uid=dtr_d38be5812b7a) of depth 5 w